# Libraries

In [4]:
import pyodbc
import pandas as pd
from openpyxl import load_workbook

# Database settings

In [30]:
# Connect to the database
server = 'DESKTOP-HTHQ40O\SQLEXPRESS'
database = 'SouthAfrica_CrimeStats'
username = 'cnoltehj'
password = 'HeidiJoy@1_2024'


# Validate Policestation and Exclude rows

In [70]:
# Load the Excel file
file_path_q1_2019_2023 = "./Dataset_CSV_Files/2023_2024/2023-2024_1st_Quarter_WEB.xlsx"

file_path_q2_2019_2023 = "./Dataset_CSV_Files/2023_2024/2023-2024_2nd_Quarter_WEB.xlsx"

file_path_q3_2019_2023 = "./Dataset_CSV_Files/2023_2024/2023-2024_3rd_Quarter_WEB.xlsx"

file_path_q4_2019_2023 = "./Dataset_CSV_Files/2023_2024/2022-2023-Q4-crime-stats.xlsx"

wb = load_workbook(file_path_q1_2019_2023, data_only=True)

# Access the desired sheet
sheet = wb["Crime stats per component"]

# Check the value of cell Q8
cell_l8 = sheet["L8"].value

# Check the value of cell Q9
cell_l9 = sheet["L9"].value

excluded_rows = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 23, 28, 36, 40, 47, 52, 54, 55, 61, 62]
excluded_columns = [1, 2, sheet.max_column - 1, sheet.max_column]  # First 2 columns and last 2 columns

print("Province : ",cell_l8)
print("Police Station : ",cell_l9)

    # Iterate over all rows except for the excluded ones
for row_number in range(2, sheet.max_row + 1):
    if row_number not in excluded_rows:
            # Retrieve row values and exclude specified columns
        row_values = [cell.value for col_num, cell in enumerate(sheet[row_number]) if col_num + 1 not in excluded_columns]

            # Print the row values as a single dataset
        print(row_values)



Province :  Western Cape
Police Station :  Calitzdorp
['Murder', 2, 0, 1, 1, 1, 0, 0]
['Sexual offences', 4, 2, 0, 7, 5, -2, -0.2857142857142857]
['Attempted murder', 0, 1, 0, 1, 2, 1, 1]
['Assault with the intent to inflict grievous bodily harm', 17, 2, 15, 13, 18, 5, 0.3846153846153846]
['Common assault', 28, 23, 28, 25, 21, -4, -0.16000000000000003]
['Common robbery', 0, 2, 4, 2, 2, 0, 0]
['Robbery with aggravating circumstances', 2, 3, 4, 1, 1, 0, 0]
['Contact crime (Crimes against the person)', 53, 33, 52, 50, 50, 0, 0]
['Rape', 1, 2, 0, 2, 2, 0, 0]
['Sexual assault', 3, 0, 0, 5, 1, -4, -0.8]
['Attempted sexual offences', 0, 0, 0, 0, 2, 2, '2 counts higher']
['Contact sexual offences', 0, 0, 0, 0, 0, 0, '0 count']
['Carjacking', 0, 0, 0, 0, 0, 0, '0 count']
['Robbery at residential premises', 1, 0, 1, 0, 1, 1, '1 count higher']
['Robbery at non-residential premises', 0, 3, 2, 0, 0, 0, '0 count']
['TRIO Crime', 1, 3, 3, 0, 1, 1, '1 count higher']
['Robbery of cash in transit', 0, 0

# Insert Stats 2019 -2023 into Database

In [71]:
# Define the province parameter
selected_province = "ZA.WC"
selected_policestation = "CZ16WC"
selected_quarter = 1

cnxn = pyodbc.connect(f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}')
cursor = cnxn.cursor()

# Iterate over all rows except excluded ones
for row_number in range(3, sheet.max_row + 1):  # Start from the third row
    
    if row_number not in excluded_rows:
        # Retrieve row values and exclude specified columns
        row_values = [cell.value for col_num, cell in enumerate(sheet[row_number]) if col_num + 1 not in excluded_columns]
        
        # Ensure row_values has enough elements before accessing
        #if len(row_values) > 1:
            # Convert the elements to int if they represent numeric values
        for i in range(1, 1):
            try:
                    row_values[i] = int(row_values[i])
            except (ValueError, IndexError):
                    # Handle the case where the value cannot be converted to int or index error occurs
                pass  # Handle this case as per your requirement
                
            # Inside the loop DB
        try:
                cursor.execute("EXEC sp_Add_QuarterlyCrimeStats ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?",
                   (str(row_values[0]).strip(), row_values[1], row_values[2], row_values[3], row_values[4], row_values[5], row_values[6], str(row_values[7]).strip(), str(selected_province).strip(), str(selected_policestation).strip(), int(selected_quarter)))
                print('Input 1')
                
                # Commit the changes to the database
                cnxn.commit()

                # Print row_values for debugging
                print("Row values:", row_values)

        except pyodbc.Error as e:
                print("Error executing SQL query:", e)


# Close the cursor and connection
cursor.close()
cnxn.close()

Input 1
Row values: ['Murder', 2, 0, 1, 1, 1, 0, 0]
Input 1
Row values: ['Sexual offences', 4, 2, 0, 7, 5, -2, -0.2857142857142857]
Input 1
Row values: ['Attempted murder', 0, 1, 0, 1, 2, 1, 1]
Input 1
Row values: ['Assault with the intent to inflict grievous bodily harm', 17, 2, 15, 13, 18, 5, 0.3846153846153846]
Input 1
Row values: ['Common assault', 28, 23, 28, 25, 21, -4, -0.16000000000000003]
Input 1
Row values: ['Common robbery', 0, 2, 4, 2, 2, 0, 0]
Input 1
Row values: ['Robbery with aggravating circumstances', 2, 3, 4, 1, 1, 0, 0]
Input 1
Row values: ['Contact crime (Crimes against the person)', 53, 33, 52, 50, 50, 0, 0]
Input 1
Row values: ['Rape', 1, 2, 0, 2, 2, 0, 0]
Input 1
Row values: ['Sexual assault', 3, 0, 0, 5, 1, -4, -0.8]
Input 1
Row values: ['Attempted sexual offences', 0, 0, 0, 0, 2, 2, '2 counts higher']
Input 1
Row values: ['Contact sexual offences', 0, 0, 0, 0, 0, 0, '0 count']
Input 1
Row values: ['Carjacking', 0, 0, 0, 0, 0, 0, '0 count']
Input 1
Row values: [

# Get Extra years for 2016 sheets and exclude rows

In [18]:
# Load the Excel file

file_path_q1_2016_2018 = "./Dataset_CSV_Files/2020_2021/first_quarter 2020_2021_crime_statistics.xlsx"

file_path_q2_2016_2018 = "./Dataset_CSV_Files/2020_2021/crime_statistics_july_september_2020_21.xlsx"

file_path_q3_2016_2018 = "./Dataset_CSV_Files/2020_2021/crime_statistics_third_qaurter 2020_2021.xlsx"

file_path_q4_2016_2018 = "./Dataset_CSV_Files/2020_2021/crime_statistics_fourth_qaurter 2020_2021_current.xlsx"

wb = load_workbook(file_path_q1_2016_2018, data_only=True)

# Access the desired sheet
sheet = wb["Dropdown tables stations"]

# Check the value of cell Q8
cell_j9 = sheet["J9"].value

# Check the value of cell Q9
cell_j10 = sheet["J10"].value

excluded_rows = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 22, 28, 29, 36, 40, 47, 53]
excluded_columns = [1, 2, sheet.max_column - 1, sheet.max_column]  # First 2 columns and last 2 columns

print("Province : ",cell_j9)
print("Police Station : ",cell_j10)

    # Iterate over all rows except for the excluded ones
for row_number in range(2, sheet.max_row + 1):
    if row_number not in excluded_rows:
            # Retrieve row values and exclude specified columns
        row_values = [cell.value for col_num, cell in enumerate(sheet[row_number]) if col_num + 1 not in excluded_columns]

            # Print the row values as a single dataset
        print(row_values)
     

Province :  KWAZULU_NATAL
Police Station :  WINTERTON
['Murder', 0, 3, 1, 0, 1, 1, '1 Case Higher', None, None, None, None, 'ALEXANDRIA', 'BAYSWATER', 'ATTERIDGEVILLE', 'BAYVIEW', 'BOLOBEDU', 'BALFOUR TVL', 'BETHANIE', 'BATLHAROS', 'BARRYDALE', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
['Sexual Offences', 4, 3, 3, 3, 1, -2, -0.6666666666666667, None, None, None, None, 'ALGOAPARK', 'BETHLEHEM', 'BEDFORDVIEW', 'BELLAIR', 'BOTLOKWA', 'BARBERTON', 'BIESIESVLEI', 'BELMONT', 'BEAUFORT WEST', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
['Attempted murder', 1, 1, 2, 4, 0, -4, -1, None, None, None, None, 'ALICE', 'BETHULIE', 'BEKKERSDAL', 'BEREA', 'BULGERIVIER', 'BELFAST', 'BLOEMHOF', 'BOETSAP', 'BELHAR', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
['Assault with the i

# Insert Extra years into database

In [ ]:
import pyodbc
from openpyxl import load_workbook

# Connect to the database
server = 'DESKTOP-HTHQ40O\SQLEXPRESS'
database = 'SouthAfrica_CrimeStats'
username = 'cnoltehj'
password = 'HeidiJoy@1_2024'
cnxn = pyodbc.connect(f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}')
cursor = cnxn.cursor()

# Define the province parameter
selected_province = "ZA.WC"
selected_policestation = "AL01WC"
selected_quarter = 1

# Iterate over all rows except excluded ones
for row_number in range(3, sheet.max_row + 1):  # Start from the third row
    
    if row_number not in excluded_rows:
        # Retrieve row values and exclude specified columns
        row_values = [cell.value for col_num, cell in enumerate(sheet[row_number]) if col_num + 1 not in excluded_columns]
        
        # Ensure row_values has enough elements before accessing
        #if len(row_values) > 1:
            # Convert the elements to int if they represent numeric values
        for i in range(1, 1):
            try:
                    row_values[i] = int(row_values[i])
            except (ValueError, IndexError):
                    # Handle the case where the value cannot be converted to int or index error occurs
                pass  # Handle this case as per your requirement
                
            # Inside the loop
        try:
                cursor.execute("EXEC sp_Add_QuarterlyCrimeStats ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?",
                   (str(row_values[0]).strip(), row_values[1], row_values[2], row_values[3], row_values[4], row_values[5], row_values[6], str(row_values[7]).strip(), str(selected_province).strip(), str(selected_policestation).strip(), int(selected_quarter)))
                

                # Commit the changes to the database
                cnxn.commit()

                # Print row_values for debugging
                print("Row values:", row_values)

        except pyodbc.Error as e:
                print("Error executing SQL query:", e)


# Close the cursor and connection
cursor.close()
cnxn.close()